In [1]:
# importing required pakcages
import pandas as pd
from tqdm import tqdm

In [16]:
ds_name = 'Epinion'
ds_rate = 5

In [4]:
dataset = pd.read_csv(f"datasets/{ds_name}/ratings_data.txt", sep=" ", names=['uid', 'iid', 'rating'])

In [5]:
dataset.head()

,uid,iid,rating
0,1,100,4
1,1,101,5
2,1,102,3
3,1,10,3
4,1,103,5


In [6]:
# what columns or attributes the dataset is included?
dataset.columns

Index(['uid', 'iid', 'rating'], dtype='object')

In [8]:
# statistics on explicit dataset
print("Original dataset statistics: ")
print(f"> No. of users: {len(dataset['uid'].unique())}")
print(f"> No. of Items: {len(dataset['iid'].unique())}")
print(f"> No. of Interaction: {dataset.shape[0]}")

Original dataset statistics: 
> No. of users: 40163
> No. of Items: 139738
> No. of Interaction: 664824


In [9]:
# checking the number of times in which a rating happened - (0: implicit, 1-10: explicit)
dataset['rating'].value_counts()

5    301053
4    194340
3     75525
2     50678
1     43228
Name: rating, dtype: int64

In [10]:
# This method return a dataframe in which the specified values are removed from a specific column. We can use it to generate the implicit/explicit dataset.
# To do this, we can remove the 0 or 1-10 values from the `Book-Rating` column.
def filter_rows_by_values(df, col, values):
    return df[~df[col].isin(values)]

In [12]:
# To remove the users with fewer than 5 interaction we first count the number of interactino per user and add a new column (`Count`) in the dataframe.
# This column shows the number of interaction per user in the dataset
users_counts = dataset['uid'].value_counts()
users_counts = users_counts.to_dict() #converts to dictionary
dataset['count'] = dataset['uid'].map(users_counts)

### Iteratively remove users and items with fewer than `ds_rate` interactions

In [18]:
user_interaction, item_interaction = 1, 1

while user_interaction != 0 or item_interaction != 0:
    print(f"The current number of user and item with < {ds_rate} interactions: ")
    # user side fewer than ds_rate cheking
    uid_value_counts = dataset['uid'].value_counts()
    user_interaction = uid_value_counts[uid_value_counts < ds_rate].count()
    print(f"No. of users < {ds_rate} ineractions: {user_interaction}")

    users_counts = dataset['uid'].value_counts()
    users_counts = users_counts.to_dict() #converts to dictionary
    dataset['count'] = dataset['uid'].map(users_counts)

    dataset = filter_rows_by_values(dataset, "count", list(range(ds_rate)))

    # item side fewer than ds_rate cheking
    bid_value_counts = dataset['iid'].value_counts()
    item_interaction = bid_value_counts[bid_value_counts < ds_rate].count()
    print(f"No. of items < {ds_rate} ineractions: {item_interaction}")

    items_counts = dataset['iid'].value_counts()
    items_counts = items_counts.to_dict() #converts to dictionary
    dataset['count'] = dataset['iid'].map(items_counts)

    dataset = filter_rows_by_values(dataset, "count", list(range(ds_rate)))

The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 16910
No. of items < 5 ineractions: 114742
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 2642
No. of items < 5 ineractions: 626
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 83
No. of items < 5 ineractions: 17
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 6
No. of items < 5 ineractions: 3
The current number of user and item with < 5 interactions: 
No. of users < 5 ineractions: 0
No. of items < 5 ineractions: 0


In [19]:
# statistics on 5 rate explicit dataset (after pre-processing)
print(f"No. of users: {len(dataset['uid'].unique())}")
print(f"No. of Items: {len(dataset['iid'].unique())}")
print(f"No. of Interaction: {dataset.shape[0]}")

No. of users: 20397
No. of Items: 21901
No. of Interaction: 446892


In [20]:
# Before we save the preprocessed explicit dataset (5Rate) we first remove the added column which is `Count`
del dataset['count']

### User and Item ID Mapping

In [21]:
uid_to_index = dict()
iid_to_index = dict()

last_user_id = 0
last_item_id = 0

for eachline in tqdm(dataset.iterrows()):
  # add a new user id with an index
  if eachline[1][0] not in uid_to_index.keys():
    uid_to_index[eachline[1][0]] = last_user_id
    last_user_id += 1
  # add a new book id with an index
  if eachline[1][1] not in iid_to_index.keys():
    iid_to_index[eachline[1][1]] = last_item_id
    last_item_id += 1

446892it [00:49, 8997.30it/s]


In [23]:
# write the file with new mapped indices into a txt file
mapped_dataset = open(f"datasets/{ds_name}/Epinion_data.csv", 'w')
mapped_dataset_txt = open(f"datasets/{ds_name}/Epinion_data.txt", 'w')
mapped_dataset.write('uid' + ',' + 'bid' + ',' + 'rating' + '\n')

for eachline in tqdm(dataset.iterrows()):
    mapped_dataset.write(str(uid_to_index[eachline[1][0]]) + "," + str(iid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
    mapped_dataset_txt.write(str(uid_to_index[eachline[1][0]]) + "," + str(iid_to_index[eachline[1][1]]) + "," + str(eachline[1][2]) + "\n")
mapped_dataset.close()

446892it [01:22, 5389.67it/s]
